## Ficha de Blast com BioPython

In [1]:
from Bio.Blast import NCBIXML 
from Bio.Blast import NCBIWWW 
from Bio import SeqIO

### Parte I

1. Grave o ficheiro “interl.fasta” no seu ambiente de trabalho. Carregue a sequência de proteína que contém (no formato Fasta). Verifique que esta tem 178 aminoácidos.

In [2]:
record = SeqIO.read(open("interl.fasta"), format="fasta") 
print (len(record.seq))

178


2. Usando as funções do BioPython efetue uma pesquisa por sequências similares no Blast, usando a base de dados “nr” (non-redundant). Grave o resultado anterior num ficheiro XML.

In [3]:
result_handle = NCBIWWW.qblast("blastp", "nr", record.format("fasta"))

In [4]:
save_file = open("interl-blast.xml", "w")
save_file.write(result_handle.read()) 
save_file.close() 
result_handle.close()

3. Realize o parsing do ficheiro anterior com as funções do BioPython. Analise os parâmetros globais da pesquisa, verificando: a base de dados usada, a matriz de substituição e os parâmetros para os espaçamentos.

In [5]:
result_handle = open("interl-blast.xml")
blast_record = NCBIXML.read(result_handle)

print ("PARAMETERS:")
print ("Database: " + blast_record.database)
print ("Matrix: " + blast_record.matrix)
print ("Gap penalties: ", blast_record.gap_penalties)

PARAMETERS:
Database: nr
Matrix: BLOSUM62
Gap penalties:  (11, 1)


4. Usando o objeto anterior, verifique o número de alinhamentos no registo. Analise com pormenor o primeiro destes alinhamentos verificando o “accession” da sequência que fez match, o id do hit, a sua definição e o número de HSPs do alinhamento. Mediante os resultados, qual é a proteína que lhe foi dada no ficheiro ?

In [6]:
print ("number hits: ", len(blast_record.alignments) )
first_alignment = blast_record.alignments[0]

print ("FIRST ALIGNMENT:")
print ("Accession: " + first_alignment.accession)
print ("Hit id: " + first_alignment.hit_id)
print ("Definition: " + first_alignment.hit_def)
print ("Alignment length: ", first_alignment.length)
print ("Number of HSPs: ", len(first_alignment.hsps))

number hits:  50
FIRST ALIGNMENT:
Accession: AAV38450
Hit id: gb|AAV38450.1|
Definition: interleukin 10, partial [synthetic construct] >gb|AAX36832.1| interleukin 10, partial [synthetic construct] >gb|AAX43106.1| interleukin 10, partial [synthetic construct] >gb|AAX43183.1| interleukin 10, partial [synthetic construct]
Alignment length:  179
Number of HSPs:  1


5. Para o HSP do alinhamento anterior, verifique: o e-value, o score, o tamanho do alinhamento, o número de caracteres iguais. Verifique o alinhamento completo.

In [7]:
hsp = first_alignment.hsps[0]
print ("E-value: ", hsp.expect)
print ("Score: ", hsp.score)
print ("Length: ", hsp.align_length)

print ("Identities: ", hsp.identities)

print (hsp.query[:90])
print (hsp.match[:90])
print (hsp.sbjct[:90])

print (hsp.query[90:])
print (hsp.match[90:])
print (hsp.sbjct[90:])


E-value:  1.81582e-129
Score:  952.0
Length:  178
Identities:  178
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFKGYLGCQALSEMIQFY
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFKGYLGCQALSEMIQFY
MHSSALLCCLVLLTGVRASPGQGTQSENSCTHFPGNLPNMLRDLRDAFSRVKTFFQMKDQLDNLLLKESLLEDFKGYLGCQALSEMIQFY
LEEVMPQAENQDPDIKAHVNSLGENLKTLRLRLRRCHRFLPCENKSKAVEQVKNAFNKLQEKGIYKAMSEFDIFINYIEAYMTMKIRN
LEEVMPQAENQDPDIKAHVNSLGENLKTLRLRLRRCHRFLPCENKSKAVEQVKNAFNKLQEKGIYKAMSEFDIFINYIEAYMTMKIRN
LEEVMPQAENQDPDIKAHVNSLGENLKTLRLRLRRCHRFLPCENKSKAVEQVKNAFNKLQEKGIYKAMSEFDIFINYIEAYMTMKIRN


6. Liste os acession numbers, as definições e os e-values dos 5 melhores alinhamentos. !

In [8]:
for i in range(5):
    alignment = blast_record.alignments[i]
    print ("Accession: " + alignment.accession)
    print ("Definition: " + alignment.hit_def)
    for hsp in alignment.hsps:
        print ("E-value: ", hsp.expect)

Accession: AAV38450
Definition: interleukin 10, partial [synthetic construct] >gb|AAX36832.1| interleukin 10, partial [synthetic construct] >gb|AAX43106.1| interleukin 10, partial [synthetic construct] >gb|AAX43183.1| interleukin 10, partial [synthetic construct]
E-value:  1.81582e-129
Accession: NP_000563
Definition: interleukin-10 isoform 1 precursor [Homo sapiens] >sp|P22301.1| RecName: Full=Interleukin-10; Short=IL-10; AltName: Full=Cytokine synthesis inhibitory factor; Short=CSIF; Flags: Precursor [Homo sapiens] >gb|AAX36387.1| interleukin 10 [synthetic construct] >gb|PSJ20588.1| hypothetical protein CVH10_16755 [Halomonas sp. ND22Bw] >emb|SJX28384.1| unnamed protein product, partial [Human ORFeome Gateway entry vector] >gb|AAA63207.1| interleukin 10 [Homo sapiens] >gb|AAA80104.1| interleukin 10 [Homo sapiens]
E-value:  2.65951e-129
Accession: AKI71928
Definition: IL10, partial [synthetic construct]
E-value:  6.75998e-129
Accession: NP_001129092
Definition: interleukin-10 precurso

7. Identifique as espécies de cada uma dessas sequências

In [9]:
# collect organisms
import re
specs = []
for i in range(5):
    alignment = blast_record.alignments[i]
    definition = alignment.hit_def
    x = re.search("\[(.*?)\]", definition).group(1)
    specs.append(x)

print ("Organisms:")
for s in specs: print(s)

Organisms:
synthetic construct
Homo sapiens
synthetic construct
Pan troglodytes
Gorilla gorilla gorilla


### Parte II

1. Considere o ficheiro “O14727.fasta” que contém a sequência da proteína humana APAF (UniProt APAF_HUMAN; O14727). Carregue o mesmo em formato Fasta e verifique que a proteína tem 1248 aminoácidos.

In [10]:
seqrecord = SeqIO.read(open("O14727.fasta"), format="fasta") 
print (len(seqrecord.seq))

1248


2. Usando as funções do BioPython efetue uma pesquisa por sequências similares no Blast, usando a base de dados “swissprot”. Grave o resultado anterior num ficheiro XML.

In [11]:
result_handle = NCBIWWW.qblast("blastp", "swissprot", seqrecord.format("fasta"))

In [12]:
save_file = open("apaf-blast-sp.xml", "w")
save_file.write(result_handle.read()) 
save_file.close() 
result_handle.close()

In [13]:
result_handle = open("apaf-blast-sp.xml")
record = NCBIXML.read(result_handle)

3. Realize o parsing do ficheiro anterior com as funções do BioPython. Analise os parâmetros globais da pesquisa, verificando: a base de dados usada, a matriz de substituição e os parâmetros para os espaçamentos.

In [14]:
print ("PARAMETERS:")
print ("Database: " + record.database)
print ("Matrix: " + record.matrix)
print ("Gap penalties: ", record.gap_penalties)

PARAMETERS:
Database: swissprot
Matrix: BLOSUM62
Gap penalties:  (11, 1)


4. Identifique a lista de accession numbers, os e-values e os comprimentos do alinhamento para os alinhamentos retornados pela pesquisa.

In [15]:
nhits = len(record.alignments) 
print ("number hits: ", nhits)

number hits:  50


In [16]:
res = []
for alignment in record.alignments:
    evalue = alignment.hsps[0].expect
    accession = alignment.accession
    leng = alignment.hsps[0].align_length
    res.append(accession + " - " + str(evalue) + " length:" + str(leng) )

print("E-values and length of alignments:")
for s in res: print(s)

E-values and length of alignments:
O14727 - 0.0 length:1248
Q9EPV5 - 0.0 length:1249
O88879 - 0.0 length:1249
Q9I9H8 - 0.0 length:1260
Q8YRI1 - 7.65095e-65 length:618
Q8YV57 - 7.4713e-62 length:630
Q8YTC2 - 1.46192e-61 length:1137
Q55563 - 2.84501e-55 length:665
Q00808 - 6.64114e-45 length:479
Q9M2Z2 - 1.26219e-33 length:310
P49695 - 2.71321e-31 length:290
A8NEG8 - 2.81379e-29 length:281
Q9V3J8 - 1.20045e-27 length:311
P61964 - 3.82246e-27 length:311
Q2KIG2 - 3.82246e-27 length:311
A0CH87 - 1.52653e-26 length:359
A0DB19 - 1.78729e-26 length:359
Q3Y8L7 - 2.07166e-26 length:303
B7PS00 - 2.49951e-26 length:307
Q54KL5 - 3.24223e-26 length:310
F6ZT52 - 3.72082e-26 length:300
C3XVT5 - 5.38307e-26 length:307
Q5M786 - 7.34739e-26 length:311
Q8W1K8 - 7.44512e-25 length:168
B8PD53 - 1.60605e-24 length:294
Q9D7H2 - 1.69689e-24 length:311
P25387 - 2.95138e-24 length:370
C4Q0P6 - 3.04259e-24 length:307
A2QP30 - 4.0154e-24 length:280
D3Z7A5 - 4.86736e-24 length:305
Q5BK30 - 5.90473e-24 length:305
B8

5. Repita o processo de Blast mas agora restringindo a pesquisa ao organismo Saccharomyces cerevisiae (na função qblast use o argumento entrez_query = "Saccharomyces cerevisiae[organism]" ) e grave os resultados.

In [17]:
result_handle2 = NCBIWWW.qblast("blastp", "swissprot", seqrecord.format("fasta"), entrez_query = "Saccharomyces cerevisiae[organism]" )

save_file = open("apaf-blast-sc.xml", "w")
save_file.write(result_handle2.read()) 
save_file.close() 
result_handle2.close()

6. Analise os resultados em 5, nomeadamente o alinhamento com menor e-value (primeiro resultado da lista). Identifique o accession (registo da Uniprot da sequência obtida), a sua descrição e o número de HSPs. Verifique o alinhamento obtido para cada HSP, verificando também o início do HSP na query e na sequência (sbjct). Note que na estrutura de um HSP tem acesso aos atributos query_start e sbjct_start.

In [18]:
result_handle2 = open("apaf-blast-sc.xml")
record2 = NCBIXML.read(result_handle2)

first_alignment = record2.alignments[0]

print ("Accession: " + first_alignment.accession)
print ("Hit id: " + first_alignment.hit_id)
print ("Definition: " + first_alignment.hit_def)

print ("Number of HSPs: ", len(first_alignment.hsps))

for hsp in first_alignment.hsps:
    print ("E-value: ", hsp.expect)
    print ("Length: ", hsp.align_length)
    print ("Identities: ", hsp.identities)
    print ("Query start: ", hsp.query_start)
    print ("Sbjct start: ", hsp.sbjct_start)
    print (hsp.query[0:90])
    print (hsp.match[0:90])
    print (hsp.sbjct[0:90])
    print ("")

Accession: P25382
Hit id: sp|P25382.3|
Definition: RecName: Full=Ribosome assembly protein 4; AltName: Full=Notchless protein homolog 1; AltName: Full=Ribosome biogenesis factor RSA4 [Saccharomyces cerevisiae S288C]
Number of HSPs:  5
E-value:  8.81809e-25
Length:  346
Identities:  86
Query start:  629
Sbjct start:  158
RIASCGADKTLQVFKAETGEKLLEIKAHEDEVLCCAFSTDDRFIATCSVDKKVKIWNSMTGE-LVHTYDEHSEQVNC-----CHFTNSSH
R+ +   D T +++  +T   +  +K H + VLC ++S D   IAT S+D  +++W+  +G+ L      HS+ +        H      
RMVTGAGDNTARIWDCDTQTPMHTLKGHYNWVLCVSWSPDGEVIATGSMDNTIRLWDPKSGQCLGDALRGHSKWITSLSWEPIHLVKPGS

E-value:  2.72376e-11
Length:  220
Identities:  60
Query start:  610
Sbjct start:  315
VVRPHTDAVYHACFSED-GQRIASCGADKTLQVFKAETGEKLLEIKAHEDEVLCCAFSTDDRFIATCSVDKKVKIWNSM-TGELVHTYDE
+++ H   V H   S D   RI +            E  +K LE      E +C      +  + T S D  + +WN + + + +     
ILKSHAHWVNHLSLSTDYALRIGAFDHTGKKPSTPEEAQKKALE----NYEKICKKNGNSEEMMVTASDDYTMFLWNPLKSTKPIARMTG

E-value:  6.9812e-11
Length:  331
Identitie

7. Analisando com atenção os resultados anteriores, bem como consultando o registo Uniprot da sequência analisada em 6 (nomeadamente o seu tamanho), o que conclui em relação à existência de genes homólogos ao APAF em S. cerevisiae. Confirme a sua ideia comparando este registo Uniprot com o da query e verificando as funções das duas proteínas.

Note-se que a sequência não tem homólogos: alinhamentos são em apenas em parte das sequências em domínios que não definem a função principal da proteina humana. Note-se as diferencas entre as duas em termos de tamanho e na função (vendo registos uniprot)